# Installs 💾

In [ ]:
!pip install ivy-core
!pip install torch
!pip install tensorflow
!pip install jax
!pip install dm-haiku
!pip install numpy

# Imports 🛃

In [43]:
import ivy

# Ivy as a Unified ML Framework 🔀

Ivy is a unified machine learning framework that aims to provide a single interface for working with various machine learning libraries, such as Numpy, TensorFlow, PyTorch, and Jax. With Ivy, you can use the same code to build and train machine learning models, regardless of the underlying library being used. All you have to do is to change one line of code 😉

## Change frameworks by one line of code ☝

With Ivy, you can define your data and operations just once and easily switch between different frameworks. To do this, simply write your operations in Ivy and use the `ivy.set_framework()` function to change the underlying framework.

P.S. there are some more advanced ways of handling backend frameworks in Ivy, so check it out in our [Deep Dive](https://lets-unify.ai/ivy/deep_dive/backend_setting.html).

### No need to worry about data types 🎨

Firstly, let's set the backend to Tensorflow

In [44]:
ivy.set_framework('tensorflow')

In [45]:
x = ivy.array([1, 2, 3])
y = ivy.array([4, 5, 6])
print((type(ivy.to_native(x))))
print(ivy.stack((x, y)))

<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)


Now let's try exactly the same code, but change the used backend framework to Pytorch.

In [46]:
ivy.set_framework('torch')

In [47]:
x = ivy.array([1, 2, 3])
y = ivy.array([4, 5, 6])
print((type(ivy.to_native(x))))
print(ivy.stack((x, y)))

<class 'torch.Tensor'>
tensor([[1, 2, 3],
        [4, 5, 6]])


You can see that defined Ivy arrays have either tf.Tensor or torch.Tensor types underneath it without any need to worry about their types.

### No need to worry about framework differences 💱

By saying that framework can be changed by just one line of code, we really mean it 🙂! By using Ivy as an ML framework, you do not need to worry about different function namings in different frameworks.

Take a clip by value operator as an example. It performs the same operation across frameworks, but has different name and argument names.
Numpy:
```
np.clip(a, a_min, a_max, out=None)
```
Tensforflow:
```
tf.clip_by_value(t, clip_value_min, clip_value_max, name=None)
```
Pytorch:
```
torch.clamp(input, min=None, max=None, *, out=None)
```
Jax:
```
jax.numpy.clip(a, a_min=None, a_max=None, out=None)
```

Here are some examples

In [48]:
import tensorflow as tf
t = tf.constant([[-10., -1., 0.], [0., 2., 10.]])
print(tf.clip_by_value(t, clip_value_min=-1, clip_value_max=1))

import numpy as np
n = np.array([[-10., -1., 0.], [0., 2., 10.]])
print(np.clip(n, a_min=-1, a_max=1))

tf.Tensor(
[[-1. -1.  0.]
 [ 0.  1.  1.]], shape=(2, 3), dtype=float32)
[[-1. -1.  0.]
 [ 0.  1.  1.]]


Ivy allows you not to worry about such things. Now let's do the same solely in Ivy.

In [49]:
ivy.set_framework('numpy')
i = ivy.array([[-10., -1., 0.], [0., 2., 10.]])
ivy.clip(i, -1, 1)

array([[-1., -1.,  0.],
       [ 0.,  1.,  1.]])

In [50]:
ivy.set_framework('torch')
i = ivy.array([[-10., -1., 0.], [0., 2., 10.]])
ivy.clip(i, -1, 1)

tensor([[-1., -1.,  0.],
        [ 0.,  1.,  1.]])

In [51]:
ivy.set_framework('tensorflow')
i = ivy.array([[-10., -1., 0.], [0., 2., 10.]])
ivy.clip(i, -1, 1)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-1., -1.,  0.],
       [ 0.,  1.,  1.]], dtype=float32)>

In [93]:
ivy.set_framework('jax')
i = ivy.array([[-10., -1., 0.], [0., 2., 10.]])
ivy.clip(i, -1, 1)

DeviceArray([[-1., -1.,  0.],
             [ 0.,  1.,  1.]], dtype=float32)

As you see, the only line that changed here is `ivy.set_framework()`.

### Unifying them all! 🍲

Finally, functions defined in Ivy are framework agnostic.
In the example below we show how Ivy's concatenation function is compatible with tensors from different frameworks. This is the same for all Ivy functions. They can accept tensors from any framework and return the correct result.

In [ ]:
import jax.numpy as jnp
import tensorflow as tf
import numpy as np
import torch

import ivy

jax_concatted   = ivy.concat((jnp.ones((1,)), jnp.ones((1,))), -1)
tf_concatted    = ivy.concat((tf.ones((1,)), tf.ones((1,))), -1)
np_concatted    = ivy.concat((np.ones((1,)), np.ones((1,))), -1)
torch_concatted = ivy.concat((torch.ones((1,)), torch.ones((1,))), -1)

We hope that this short demo gives you a better understanding of basic Ivy functionality and got your interest in learning more about Ivy!

# Ivy as a standalone ML framework 🌀

Finally, let's train a simple two layer network using Ivy.

### Set Backend Framework

You can change the framework to any of the following: `torch`, `tensforflow`, or `jax`.

In [85]:
ivy.set_framework('torch')

### Define Model

In [86]:
class MyModel(ivy.Module):
    def __init__(self):
        self.linear0 = ivy.Linear(3, 64)
        self.linear1 = ivy.Linear(64, 1)
        ivy.Module.__init__(self)

    def _forward(self, x):
        x = ivy.relu(self.linear0(x))
        return ivy.sigmoid(self.linear1(x))

### Create Model

In [87]:
model = MyModel()

### Create Optimizer

In [88]:
optimizer = ivy.Adam(1e-4)

### Input and Target

In [89]:
x_in = ivy.array([1., 2., 3.])
target = ivy.array([0.])

### Loss Function

In [90]:
def loss_fn(v):
    out = model(x_in, v=v)
    return ivy.reduce_mean((out - target)**2)[0]

### Training Loop

In [91]:
for step in range(100):
    loss, grads = ivy.execute_with_gradients(loss_fn, model.v)
    model.v = optimizer.step(model.v, grads)
    print('step {} loss {}'.format(step, ivy.to_numpy(loss).item()))

print('Finished training!')

step 0 loss 0.49040043354034424
step 1 loss 0.48975786566734314
step 2 loss 0.4892795979976654
step 3 loss 0.48886892199516296
step 4 loss 0.4884953498840332
step 5 loss 0.4881443977355957
step 6 loss 0.4878086447715759
step 7 loss 0.48748287558555603
step 8 loss 0.48716384172439575
step 9 loss 0.48684927821159363
step 10 loss 0.48653748631477356
step 11 loss 0.48622724413871765
step 12 loss 0.4859171509742737
step 13 loss 0.48560672998428345
step 14 loss 0.48529526591300964
step 15 loss 0.4849821627140045
step 16 loss 0.48466697335243225
step 17 loss 0.4843493402004242
step 18 loss 0.4840289056301117
step 19 loss 0.4837053418159485
step 20 loss 0.4833785891532898
step 21 loss 0.4830484390258789
step 22 loss 0.48271444439888
step 23 loss 0.48237672448158264
step 24 loss 0.48203518986701965
step 25 loss 0.48168954253196716
step 26 loss 0.4813397228717804
step 27 loss 0.4809857904911041
step 28 loss 0.48062753677368164
step 29 loss 0.48026490211486816
step 30 loss 0.479898065328598
step 

In [92]:
loss_fn(model.v)

tensor(0.4439, grad_fn=<SelectBackward0>)